In [4]:
import pandas as pd
import numpy as np

SUB_PATH = "./submissions/"
DATA_PATH = "../../data/raw/"
OOF_PATH = "./oof_data/"

# 검증된 m1, m2를 80%로 배치하여 하한선을 방어하고, m5로 상한선을 치기.
# 이번에는 m4를 과감히 제외하여 '정합성'을 최우선으로 복구.
weights = [0.4, 0.4, 0.2] 

# 1. 모델 로드
m1 = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted']             # 0.78116
m2 = pd.read_csv(f"{SUB_PATH}Pure_0781_AUC_0.77322.csv")['voted'] # 0.78108
m5 = pd.read_csv(f"{SUB_PATH}m5_FE_test_preds.csv")['voted']     # 0.77353 (CV)

# 2. Rank Averaging (순위 정규화)
# 확률값이 아닌 '상대적 순위'를 사용하여 0.71 같은 폭락을 방지.
m1_rank = m1.rank(pct=True)
m2_rank = m2.rank(pct=True)
m5_rank = m5.rank(pct=True)

# 3. 가중치 합산
final_rank_blend = (m1_rank * weights[0]) + (m2_rank * weights[1]) + (m5_rank * weights[2])

# 4. 제출 파일 생성
submission = pd.read_csv(DATA_PATH + "sample_submission.csv")
submission['voted'] = final_rank_blend
submission.to_csv(f"{SUB_PATH}23_Safety_Recovery_Rank_82.csv", index=False)

In [5]:
print(m1.head())
print(m5.head())

0    1.493283
1    1.961895
2    1.473205
3    1.144872
4    1.664195
Name: voted, dtype: float64
0    0.404315
1    0.072379
2    0.500393
3    0.804772
4    0.273232
Name: voted, dtype: float64


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# 1. 정답 및 인덱스 준비 (m5와 동일한 필터링 적용)
df = pd.read_csv(DATA_PATH + "train.csv")
df_filtered = df[df['familysize'] <= 50].reset_index(drop=True)
a_cols = [col for col in df_filtered.columns if col.endswith('A') and col.startswith('Q')]
mask = df_filtered[a_cols].std(axis=1) > 0
valid_indices = df_filtered[mask].index

train_y = (2 - df_filtered['voted'].to_numpy()).astype(np.float32)[valid_indices]

# 2. OOF 로드 및 Rank 변환
m1_oof = np.load(OOF_PATH + "exp11_sota_AUC_0.77212.npy")[valid_indices]
m2_oof = np.load(OOF_PATH + "exp12_auc_AUC_0.77233.npy")[valid_indices]
m5_oof = np.load(OOF_PATH + "exp20_m5_FE_AUC_0.77353.npy") # 이미 필터링된 경우

# 중요: m1, m2는 '2-prob' 형태일 수 있으므로 순위를 매길 때 방향을 맞춰야 합니다.
# m1, m2 값이 작을수록 1에 가까우면 그대로 rank, 클수록 1에 가까우면 -rank
# 여기서는 안전하게 모든 oof를 순위화합니다.
m1_rank = pd.Series(m1_oof).rank(pct=True)
m2_rank = pd.Series(m2_oof).rank(pct=True)
m5_rank = pd.Series(m5_oof).rank(pct=True)

# 3. 시뮬레이션: 8:2 가중치 앙상블 (m1:0.4, m2:0.4, m5:0.2)
ensemble_oof = (m1_rank * 0.4) + (m2_rank * 0.4) + (m5_rank * 0.2)

# 4. 검증 결과 출력
current_auc = roc_auc_score(train_y, ensemble_oof)
print("="*50)
print(f"[Exp 23] Rank Ensemble 시뮬레이션 결과")
print(f"최종 로컬 AUC: {current_auc:.5f}")
print("="*50)

# 0.77353(m5 단일)보다 높게 나온다면, 리더보드 0.782+ 복귀는 확정적입니다.

[Exp 23] Rank Ensemble 시뮬레이션 결과
최종 로컬 AUC: 0.77327
